# Configuration-Based Example

This notebook demonstrates how to use YAML configuration files for managing complex MQTT setups with the FlowerPower MQTT Plugin.

## Overview

This example shows how to:
- Create configuration files programmatically
- Load plugins from configuration files
- Manage complex subscription setups
- Save runtime configuration changes

## Prerequisites

Make sure you have:
- MQTT broker running
- Redis server running (for job queue)
- FlowerPower project set up
- Required Python packages installed

## Step 1: Import Required Libraries

Import the necessary libraries for configuration-based MQTT setup.

In [ ]:
import asyncio
import logging
from pathlib import Path
from flowerpower_mqtt import MQTTPlugin, FlowerPowerMQTTConfig

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Step 2: Create Configuration Programmatically

Create a comprehensive configuration file with MQTT settings, job queue configuration, and predefined subscriptions.

In [ ]:
def create_example_config():
    """Create an example configuration file."""
    
    config = FlowerPowerMQTTConfig()
    
    # MQTT broker settings
    config.mqtt.broker = "localhost"
    config.mqtt.port = 1883
    config.mqtt.keepalive = 60
    config.mqtt.client_id = "flowerpower_config_example"
    
    # Job queue settings
    config.job_queue.enabled = True
    config.job_queue.redis_url = "redis://localhost:6379"
    config.job_queue.queue_name = "mqtt_pipelines"
    config.job_queue.worker_count = 4
    
    # Base directory and logging
    config.base_dir = "."
    config.log_level = "INFO"
    
    # Predefined subscriptions
    from flowerpower_mqtt.config import SubscriptionConfig
    
    config.subscriptions = [
        SubscriptionConfig(
            topic="sensors/+/temperature",
            pipeline="temperature_processor",
            qos=1,
            execution_mode="async"
        ),
        SubscriptionConfig(
            topic="sensors/+/humidity", 
            pipeline="humidity_processor",
            qos=1,
            execution_mode="async"
        ),
        SubscriptionConfig(
            topic="alerts/critical",
            pipeline="critical_alert_handler",
            qos=2,
            execution_mode="sync"
        ),
        SubscriptionConfig(
            topic="logs/+/error",
            pipeline="error_log_processor", 
            qos=0,
            execution_mode="async"
        )
    ]
    
    # Save configuration
    config_file = Path("example_mqtt_config.yml")
    config.to_yaml(config_file)
    logger.info(f"Created example configuration: {config_file}")
    
    return config_file

# Create the configuration file
config_file = create_example_config()
print(f"Configuration file created: {config_file}")

## Step 3: Load Plugin from Configuration

Load the MQTT plugin using the configuration file we just created.

In [ ]:
# Load plugin from configuration
logger.info(f"Loading plugin from configuration: {config_file}")
mqtt = MQTTPlugin.from_config(config_file)
logger.info("Plugin loaded successfully from configuration!")

## Step 4: Connect to MQTT Broker

Connect to the MQTT broker using the loaded configuration.

In [ ]:
# Connect to MQTT broker
logger.info("Connecting to MQTT broker...")
await mqtt.connect()
logger.info("Connected successfully!")

## Step 5: Display Loaded Subscriptions

Show the subscriptions that were loaded from the configuration file.

In [ ]:
# Display loaded subscriptions
subscriptions = mqtt.get_subscriptions()
logger.info(f"Loaded {len(subscriptions)} subscriptions from config:")
for sub in subscriptions:
    logger.info(
        f"  - {sub['topic']} -> {sub['pipeline']} "
        f"(QoS {sub['qos']}, {sub['execution_mode']} mode)"
    )

## Step 6: Add Runtime Subscriptions

Demonstrate adding additional subscriptions programmatically at runtime.

In [ ]:
# Add a runtime subscription
await mqtt.subscribe(
    topic="runtime/+/data",
    pipeline_name="runtime_processor",
    qos=1,
    execution_mode="async"
)
logger.info("Added runtime subscription")

## Step 7: Start MQTT Listener

Start listening for MQTT messages using the configured subscriptions.

In [ ]:
# Start listener
logger.info("Starting MQTT listener. Press Ctrl+C to stop...")
await mqtt.start_listener(background=False)

## Step 8: Save Final Configuration

Save the final configuration including any runtime changes.

In [ ]:
# Save final configuration (including runtime additions)
final_config_file = Path("final_mqtt_config.yml") 
mqtt.save_config(final_config_file)
logger.info(f"Saved final configuration: {final_config_file}")

## Step 9: Clean Shutdown

Properly disconnect and clean up resources.

In [ ]:
# Clean shutdown
logger.info("Stopping MQTT plugin...")
await mqtt.disconnect()
logger.info("MQTT plugin stopped")

# Cleanup example config file
if config_file.exists():
    config_file.unlink()
    logger.info(f"Cleaned up example config: {config_file}")

## Complete Example

Here's the complete example in a single executable cell:

In [ ]:
import asyncio
import logging
from pathlib import Path
from flowerpower_mqtt import MQTTPlugin, FlowerPowerMQTTConfig

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_example_config():
    """Create an example configuration file."""
    
    config = FlowerPowerMQTTConfig()
    
    # MQTT broker settings
    config.mqtt.broker = "localhost"
    config.mqtt.port = 1883
    config.mqtt.keepalive = 60
    config.mqtt.client_id = "flowerpower_config_example"
    
    # Job queue settings
    config.job_queue.enabled = True
    config.job_queue.redis_url = "redis://localhost:6379"
    config.job_queue.queue_name = "mqtt_pipelines"
    config.job_queue.worker_count = 4
    
    # Base directory
    config.base_dir = "."
    config.log_level = "INFO"
    
    # Predefined subscriptions
    from flowerpower_mqtt.config import SubscriptionConfig
    
    config.subscriptions = [
        SubscriptionConfig(
            topic="sensors/+/temperature",
            pipeline="temperature_processor",
            qos=1,
            execution_mode="async"
        ),
        SubscriptionConfig(
            topic="sensors/+/humidity", 
            pipeline="humidity_processor",
            qos=1,
            execution_mode="async"
        ),
        SubscriptionConfig(
            topic="alerts/critical",
            pipeline="critical_alert_handler",
            qos=2,
            execution_mode="sync"
        ),
        SubscriptionConfig(
            topic="logs/+/error",
            pipeline="error_log_processor", 
            qos=0,
            execution_mode="async"
        )
    ]
    
    # Save configuration
    config_file = Path("example_mqtt_config.yml")
    config.to_yaml(config_file)
    logger.info(f"Created example configuration: {config_file}")
    
    return config_file

async def main():
    """Configuration-based MQTT plugin usage."""
    
    # Create example configuration file
    config_file = create_example_config()
    
    try:
        # Load plugin from configuration
        logger.info(f"Loading plugin from configuration: {config_file}")
        mqtt = MQTTPlugin.from_config(config_file)
        
        # Connect to MQTT broker
        logger.info("Connecting to MQTT broker...")
        await mqtt.connect()
        
        # Display loaded subscriptions
        subscriptions = mqtt.get_subscriptions()
        logger.info(f"Loaded {len(subscriptions)} subscriptions from config:")
        for sub in subscriptions:
            logger.info(
                f"  - {sub['topic']} -> {sub['pipeline']} "
                f"(QoS {sub['qos']}, {sub['execution_mode']} mode)"
            )
        
        # You can still add more subscriptions programmatically
        await mqtt.subscribe(
            topic="runtime/+/data",
            pipeline_name="runtime_processor",
            qos=1,
            execution_mode="async"
        )
        
        # Start listener
        logger.info("Starting MQTT listener. Press Ctrl+C to stop...")
        await mqtt.start_listener(background=False)
        
    except KeyboardInterrupt:
        logger.info("Received keyboard interrupt")
    except Exception as e:
        logger.error(f"Error: {e}")
    finally:
        # Clean shutdown
        logger.info("Stopping MQTT plugin...")
        if 'mqtt' in locals():
            await mqtt.disconnect()
            
            # Save final configuration (including runtime additions)
            final_config_file = Path("final_mqtt_config.yml") 
            mqtt.save_config(final_config_file)
            logger.info(f"Saved final configuration: {final_config_file}")
        
        # Cleanup example config file
        if config_file.exists():
            config_file.unlink()
            logger.info(f"Cleaned up example config: {config_file}")
            
        logger.info("MQTT plugin stopped")

# Uncomment the line below to run the complete example
# await main()